<a href="https://colab.research.google.com/github/kimhwijin/TensorflowWithKeras/blob/master/RNN/RNN_emotion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import numpy as np
import os
import shutil
import tensorflow as tf

from sklearn.metrics import accuracy_score, confusion_matrix

In [14]:
def download_and_read(url):
  local_file = url.split('/')[-1]
  local_file = local_file.replace("%20", " ")
  p = tf.keras.utils.get_file(local_file, url, extract=True, cache_dir=".")
  local_folder = os.path.join("drive/MyDrive/Datasets/UCI_ML_ES", local_file.split('.')[0])
  labeled_sentences = []
  for labeled_filename in os.listdir(local_folder):
    if labeled_filename.endswith("_labelled.txt"):
      with open(os.path.join(local_folder, labeled_filename), "r") as f:
        for line in f:
          sentence, label = line.strip().split("\t")
          labeled_sentences.append((sentence, label))

  return labeled_sentences
labeled_sentences = download_and_read(
    "https://archive.ics.uci.edu/ml/machine-learning-databases/00331/sentiment%20labelled%20sentences.zip"
)
sentences = [s for (s, l) in labeled_sentences]
labels = [l for (s, l) in labeled_sentences]

In [15]:
#문장의 각 단어를  token으로 변환하기
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(sentences)
vocab_size = len(tokenizer.word_counts)
print("vocabulary size : {:d}".format(vocab_size))

word2idx = tokenizer.word_index
idx2word = {v:k for (k, v) in word2idx.items()}

vocabulary size : 5271


In [16]:
#정적인 입력크기를 정하기 위해, 대부분 문장을 수용할 수 있을 만큼의 시퀀스 길이를 정한다.
seq_lengths = np.array([len(s.split()) for s in sentences])
print([(p, np.percentile(seq_lengths, p)) for p in [75, 80, 90, 95, 99, 100]])
#75% : 16단어 , 80% 18단어, 99% 36단어, 최대길이 71단어.

[(75, 16.0), (80, 18.0), (90, 22.0), (95, 26.0), (99, 36.0), (100, 71.0)]


In [17]:
#최대 단어길이 64로 설정
max_seqlen = 64

#데이터 셋 생성
sentences_as_ints = tokenizer.texts_to_sequences(sentences)
#남은 부분은 0으로 채워준다.
sentences_as_ints = tf.keras.preprocessing.sequence.pad_sequences(sentences_as_ints, maxlen=max_seqlen)
labels_as_ints = np.array(labels, dtype=np.float32)
dataset = tf.data.Dataset.from_tensor_slices((sentences_as_ints, labels_as_ints))

dataset = dataset.shuffle(10000)
test_size = len(sentences) // 3
val_size = (len(sentences) - test_size) // 10
test_dataset = dataset.take(test_size)
val_dataset = dataset.skip(test_size).take(val_size)
train_dataset = dataset.skip(test_size + val_size)

batch_size = 64
train_dataset = train_dataset.batch(batch_size)
val_dataset = val_dataset.batch(batch_size)
test_dataset = test_dataset.batch(batch_size)

In [18]:
print(train_dataset)

<BatchDataset shapes: ((None, 64), (None,)), types: (tf.int32, tf.float32)>


In [19]:
class SentimentAnalysisModel(tf.keras.Model):
  def __init__(self, vocab_size, max_seqlen, **kwargs):
    super(SentimentAnalysisModel, self).__init__(**kwargs)
    self.embedding = tf.keras.layers.Embedding(vocab_size, max_seqlen)
    self.bilstm = tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(max_seqlen)
    )
    self.dense = tf.keras.layers.Dense(64, activation='relu')
    self.out = tf.keras.layers.Dense(1, activation='sigmoid')

  def call(self, x):
    x = self.embedding(x)
    x = self.bilstm(x)
    x = self.dense(x)
    x = self.out(x)
    return x

model = SentimentAnalysisModel(vocab_size+1, max_seqlen)
model.build(input_shape=(batch_size, max_seqlen))
model.compile(
    loss="binary_crossentropy",
    optimizer="adam", 
    metrics=["accuracy"]
)
model.summary()

Model: "sentiment_analysis_model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      multiple                  337408    
_________________________________________________________________
bidirectional_4 (Bidirection multiple                  66048     
_________________________________________________________________
dense_8 (Dense)              multiple                  8256      
_________________________________________________________________
dense_9 (Dense)              multiple                  65        
Total params: 411,777
Trainable params: 411,777
Non-trainable params: 0
_________________________________________________________________


In [20]:
data_dir = "drive/MyDrive/Colab Notebooks/models/RNN/sentiment_analysis/"
logs_dir = "drive/MyDrive/Colab Notebooks/logs/RNN/sentiment_analysis/"
best_model_file = os.path.join(data_dir, "best_model.h5")

In [21]:
checkpoint = tf.keras.callbacks.ModelCheckpoint(best_model_file, save_weights_only=True, save_best_only=True)
tensorboard = tf.keras.callbacks.TensorBoard(log_dir=logs_dir)
num_epochs = 10
history = model.fit(train_dataset, epochs=num_epochs, 
    validation_data=val_dataset,
    callbacks=[checkpoint, tensorboard])


Epoch 1/10
29/29 [==============================] - 8s 155ms/step - loss: 0.6910 - accuracy: 0.5117 - val_loss: 0.6747 - val_accuracy: 0.6650
Epoch 2/10
29/29 [==============================] - 3s 109ms/step - loss: 0.6246 - accuracy: 0.7250 - val_loss: 0.4545 - val_accuracy: 0.8700
Epoch 3/10
29/29 [==============================] - 3s 108ms/step - loss: 0.3645 - accuracy: 0.8717 - val_loss: 0.1966 - val_accuracy: 0.9300
Epoch 4/10
29/29 [==============================] - 3s 107ms/step - loss: 0.2274 - accuracy: 0.9183 - val_loss: 0.1413 - val_accuracy: 0.9450
Epoch 5/10
29/29 [==============================] - 3s 103ms/step - loss: 0.1537 - accuracy: 0.9528 - val_loss: 0.1375 - val_accuracy: 0.9550
Epoch 6/10
29/29 [==============================] - 3s 105ms/step - loss: 0.1186 - accuracy: 0.9661 - val_loss: 0.0842 - val_accuracy: 0.9750
Epoch 7/10
29/29 [==============================] - 3s 104ms/step - loss: 0.0873 - accuracy: 0.9761 - val_loss: 0.1045 - val_accuracy: 0.9700
Epoch 

In [22]:
best_model = SentimentAnalysisModel(vocab_size+1, max_seqlen)
best_model.build(input_shape=(batch_size, max_seqlen))
best_model.load_weights(best_model_file)
best_model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)
test_loss, test_acc = best_model.evaluate(test_dataset)
print("test loss : {:.3f}, test accuracy : {:.3f}".format(test_loss, test_acc))

16/16 [==============================] - 2s 26ms/step - loss: 0.0364 - accuracy: 0.9930
test loss : 0.036, test accuracy : 0.993


In [26]:
labels, predictions = [], []
idx2word[0] = "PAD"
is_first_batch = True
for test_batch in test_dataset:
  inputs_b , labels_b = test_batch
  pred_batch = best_model.predict(inputs_b)
  predictions.extend([(1 if p > 0.5 else 0) for p in pred_batch])
  labels.extend([l for l in labels_b])
  if is_first_batch:
    for rid in range(inputs_b.shape[0]):
      words = [ idx2word[idx] for idx in inputs_b[rid].numpy()]
      words = [w for w in words if w != "PAD"]
      sentence = " ".join(words)
      print("{:f}\t{:f}\t{:s}".format(labels[rid], predictions[rid], sentence))

is_first_batch = False




1.000000	1.000000	our server was very nice and even though he looked a little overwhelmed with all of our needs he stayed professional and friendly until the end
0.000000	0.000000	why are these sad little vegetables so overcooked
0.000000	0.000000	the ri style calamari was a joke
1.000000	1.000000	overall i rate this movie a 10 out of a 1 10 scale
1.000000	1.000000	the lighting is just dark enough to set the mood
0.000000	0.000000	it was so bad i had lost the heart to finish it
0.000000	0.000000	the script looks as if only a rough draft was written and shooting began before a finished script was completed
1.000000	1.000000	it's just adorable seeing mickey playing turkey in the straw in a highly imaginative if occasionally cruel way
0.000000	0.000000	this is a bad film with bad writing and good actors an ugly cartoon crafted by paul haggis for people who can't handle anything but the bold strokes in storytelling a picture painted with crayons
1.000000	1.000000	reversible plug works grea